In [9]:
import cvxopt as cvx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.spatial.distance as dt
from sklearn.metrics import accuracy_score

In [14]:
data_X = np.genfromtxt('hw06_images.csv', delimiter = ",")
data_Y = np.genfromtxt('hw06_labels.csv',delimiter = ",")

In [15]:
train_x = data_X[:1000,:]
train_y = data_Y[:1000].astype(int)
test_x = data_X[1000:,:] 
test_y = data_Y[1000:].astype(int)

N_train = len(train_y)
N_test = len(test_y)
D_train = train_x.shape[1]
labels  = np.max(train_y);

In [16]:
# Gaussian Kernel

def gaussian_kernel(X1, X2, s):
    D = dt.cdist(X1, X2)
    K = np.exp(-D**2 / (2 * s**2))
    return(K)

In [ ]:
s = 10 #given in document
K = gaussian_kernel(train_x, train_x, s)
arr_alpha = []
arr_w0 = []
arr_train_y = []

for l in range(labels):
  label = l + 1;

  train_class = ((train_y == label).astype(int))*2 - 1 #one vs all
  yyK = np.matmul(train_class[:,None], train_class[None,:]) * K

  C = 10
  epsilon = 1e-3
  #same with lab 8

  P = cvx.matrix(yyK)
  q = cvx.matrix(-np.ones((N_train, 1)))
  G = cvx.matrix(np.vstack((-np.eye(N_train), np.eye(N_train))))
  h = cvx.matrix(np.vstack((np.zeros((N_train, 1)), C * np.ones((N_train, 1)))))
  A = cvx.matrix(1.0 * train_class[None,:])
  b = cvx.matrix(0.0)


  result = cvx.solvers.qp(P, q, G, h, A, b)
  alpha = np.reshape(result["x"], N_train)
  alpha[alpha < C * epsilon] = 0
  alpha[alpha > C * (1 - epsilon)] = C

  support, = np.where(alpha != 0)
  active, = np.where(np.logical_and(alpha != 0, alpha < C))
  w0 = np.mean(train_class[active] * (1 - np.matmul(yyK[np.ix_(active, support)], alpha[support])))

  arr_alpha.append(alpha)
  arr_w0.append(w0)
  arr_train_y.append(train_class)

In [26]:
#Actual one vs all for score 

arr_predicted = []
for t in range(labels):
  train_predicted = np.matmul(K, arr_train_y[t][:,None] * arr_alpha[t][:,None]) + arr_w0[t]
  arr_predicted.append(train_predicted)


predicted_matrix = np.hstack(arr_predicted)
predictions_train = np.argmax(predicted_matrix,axis=1) + 1

In [27]:
predictions = np.argmax(predicted_matrix,axis=1) 
predictions += 1 

In [28]:
confusion_matrix = pd.crosstab(np.reshape(predictions_train, N_train), train_y, rownames = ['y_predicted'], colnames = ['y_train'])
print(confusion_matrix)

y_train        1    2    3    4    5
y_predicted                         
1            207    1    0    9    0
2              2  199    1    1    0
3              0    1  204    6    0
4              0    1    4  185    1
5              0    0    0    0  178
